In [1]:
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy import stats
import numpy as np
import sys

%matplotlib inline
plt.rcParams['savefig.dpi'] = 300

#stroke-miterlimit change this for svg file output
%reload_ext autoreload

In [2]:
from fluctuation_code.MSS_MLE import *
#imports the code set for completing the MLE optimization as a function of m, M
#yay! completed writing this on

#function set described below

## Probability function of observing 'r' mutants in a given culture Given a rate m  $$p_0 = e^{-m}$$   $$p_r = \frac{m}{r} \sum_{i=0}^{r-1} \frac{p_i}{(r-i+1)} $$   Maximum likelihood function (this is what needs to be maximized.   $$f(r|m) = \prod_{i=1}^{c} f(r_i|m) = (p_0)^{c_0} (p_1)^{c_1} (p_2)^{c_2} (p_{r_{max}})^{c_{r_{max}}} $$    We also have the following MMS-MLE confidence intervals for the MSS-MLE method    $$CL_{+95\%} = ln(m) + 1.96 \sigma (e^{1.96 \sigma})^{-0.315}$$  $$CL_{-95\%} = ln(m) - 1.96 \sigma (e^{1.96 \sigma})^{+0.315}$$  with $$\sigma_{ln(m)} \approx \frac{1.225m^{-0.315}}{\sqrt{C}} $$

The code here is for developing the algorithm which executes automatic  MMS-MLE according to http://www.mitochondria.org/protocols/FALCOR.html 

data as described in summary 2016_11_01 (good experimental quality, etc.)

Pr_ARRAY(m, r=0, P_R=[]) #takes trial value m mutations, with r observed number and computes probability
Pr(m,r,P_R)              #non-array version of Pr_ARRAY

mut_to_array(x,r_max)    #takes x colony values and converts to a counting array/histogram (mostly sparse usually)

f_MLF(P_R, C)            #computes log of MLF function as np.log(P_R).dot(counts) 
MLF(m,Y,r_max=50)        #m is trial mutation rate (can be array), Y is mutation matrix, computes f_MLF
                            #feature selects array or float depending on input
                            #returns f_MLF value(s) for array of single m
                            
optimize_test            #returns x_1 and x_2 intervals that are 'best' for a given guess, used for finding optimum
optimizer(Y,x_guess1, x_guess2, r_max, decay, draws, mult)
                         #function takes many parameters and outputs limits, logm as values for the optimal 'm'
                         
confidence(m,cells,c=8)  #computes confidence intervals

getM(x,cells,c=8)        #ties together the package. runs optimizer on a given set of X observations
                         #returns confidence intervals and optimal 'm'


In [3]:

def FALCOR_format(counts = [1,1,1,1,2,1,1,1], cellN =35000):
    
    conf, m = getM(counts, cellN, 16, r_max=170, decay=.5)

    print('%.3f' % m)
    print('%.3f' % conf[0])
    print('%.3f' % conf[1])
    
    return

#1686, 3912,266


In [4]:
FALCOR_format([0,2,4,3,6,1,3,0], cellN = 5*1E3*23)

#15 (8.6,23)
#13 (7, 20)
#9 (4.5,15)

110.859
176.790
57.155


/home/aaroncp1an0/aaroncp1an0@gmail.com/lab_smolke/Afluctuationassay_tables/fluctuation_code/MSS_MLE.py:68: RuntimeWarning: divide by zero encountered in log
  return np.log(P_R).dot(C)


In [5]:
def T_sig(m1,m2,culture1=8, culture2=8):
    #degrees of freedom is C1+C2-2
    sigma1 = np.e**(1.225*m1**-.315/np.sqrt(culture1))
    sigma2 = np.e**(1.225*m2**-.315/np.sqrt(culture2))
    return (np.log(m1) - np.log(m2)) / np.sqrt(sigma1**2/culture1 + sigma2**2/culture2)

In [6]:
tv = T_sig(6.5,7.3, culture1=16, culture2=16)
pval = stats.t.sf(np.abs(tv), 14)*2

print('pval:   '+str(pval))
print('tval:   '+str(tv))

pval:   0.785175402994
tval:   -0.277867561746


In [36]:
FALCOR_format([16,12,5,3,1,4,3,2,21,10,8,11,13,8,2,3,3,21,4,4,1,0,14,2,1











    ],cellN=3660000
)

6.613
9.805
3.915


/home/aaroncp1an0/aaroncp1an0@gmail.com/lab_smolke/Afluctuationassay_tables/fluctuation_code/MSS_MLE.py:68: RuntimeWarning: divide by zero encountered in log
  return np.log(P_R).dot(C)


In [20]:
filename='FALCOR_input.csv'
filetext = np.genfromtxt(filename, dtype='U', delimiter=':')  #autostrip=True, )

print(filetext) #.astype('str'))

x = [int(i) for i in filetext[0]] #convert the first line to integer array
y = [np.fromstring(j,dtype=int,sep=',') for j in filetext[1]]

print('OUTPUT')
print(x)
print(y)

[['1320000' '1500000' '1560000' '1560000' '1200000' '1320000']
 ['3,6,19,4,1,5,6,4' '0,2,2,0,0,16,0,6' '3,0,2,0,0,2,3,0' '1,1,1,2,0,2,2,0'
  '2,5,6,7,12,15,4,6' '1,0,0,0,0,0,0,0']]
OUTPUT
[1320000, 1500000, 1560000, 1560000, 1200000, 1320000]
[array([ 3,  6, 19,  4,  1,  5,  6,  4]), array([ 0,  2,  2,  0,  0, 16,  0,  6]), array([3, 0, 2, 0, 0, 2, 3, 0]), array([1, 1, 1, 2, 0, 2, 2, 0]), array([ 2,  5,  6,  7, 12, 15,  4,  6]), array([1, 0, 0, 0, 0, 0, 0, 0])]


In [21]:
for i in range(len(x)):
    FALCOR_format(y[i],x[i])
    print('\n')

19.285
28.439
11.524


5.442
9.181
2.487


4.616
7.918
2.031


5.777
9.622
2.718


26.219
37.832
16.268


0.947
2.114
0.180




/home/aaroncp1an0/aaroncp1an0@gmail.com/lab_smolke/Afluctuationassay_tables/fluctuation_code/MSS_MLE.py:68: RuntimeWarning: divide by zero encountered in log
  return np.log(P_R).dot(C)


In [121]:
print('%.1f' % conf[0] + '\t123')

37817.8	123


In [12]:
y=mut_to_array(counts)
limits, m1 = optimizer(y,decay=.1,draws=70,mult=3)

plt.plot(range(len(limits[:,0])),10**limits[:,0],color='g')
plt.plot(range(len(limits[:,0])),10**limits[:,1],color='g')

limits, m2 = optimizer(y,decay=.6,draws=70,mult=3)

plt.plot(range(len(limits[:,0])),10**limits[:,0],color='b')
plt.plot(range(len(limits[:,0])),10**limits[:,1],color='b')

limits, m3 = optimizer(y,decay=.9,draws=70,mult=3)

plt.plot(range(len(limits[:,0])),10**limits[:,0],color='r')
plt.plot(range(len(limits[:,0])),10**limits[:,1],color='r')

print(m1,m2,m3)

NameError: name 'optimizer' is not defined